<a href="https://colab.research.google.com/github/Shivani-Pathak/Bluejay-Assignment/blob/main/Bluejayassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [14]:
import pandas as pd
from datetime import timedelta

def analyze_data(data, output_file):
    with open(output_file, 'w') as file:
        for name, group in data.groupby('Employee Name'):
            consecutive_days = 0
            previous_end_time = None

            for index, entry in group.iterrows():
                current_start_time = entry['Time']

                # Check for 7 consecutive days
                if consecutive_days == 7:
                    file.write(f"Employee {name} ({entry['Position ID']}) worked for 7 consecutive days.\n")
                    consecutive_days = 0

                # Check for less than 10 hours between shifts but greater than 1 hour
                if previous_end_time is not None:
                    time_difference = (current_start_time - previous_end_time).total_seconds() // 3600
                    if 1 < time_difference < 10:
                        file.write(f"Employee {name} ({entry['Position ID']}) has less than 10 hours between shifts but greater than 1 hour.\n")

                # Check for more than 14 hours in a single shift
                hours_worked_str = str(entry['Timecard Hours (as Time)'])

                try:
                    hours_worked = sum(int(x) * 60 ** i for i, x in enumerate(reversed(hours_worked_str.split(":"))))
                except ValueError:
                    # Handle the case where the value is not a valid time
                    hours_worked = 0

                if hours_worked > 14:
                    file.write(f"Employee {name} ({entry['Position ID']}) worked for more than 14 hours in a single shift.\n")

                consecutive_days += 1
                previous_end_time = current_start_time + timedelta(hours=hours_worked)

            # Check if the last shift of the week exceeds 14 hours to not  consider the edge case that arise. Its best not to make assumptions.
            if previous_end_time is not None and (previous_end_time - current_start_time).total_seconds() // 3600 > 14:
                file.write(f"Employee {name} ({entry['Position ID']}) worked for more than 14 hours in the last shift of the week.\n")

if __name__ == "__main__":
    file_path = '/content/drive/MyDrive/Colab Notebooks/Assignment_Timecard.xlsx'  # Update with your input file path
    output_file = '/content/drive/MyDrive/output.txt'   # Update with the desired output file path
    data = pd.read_excel(file_path)
    data['Time'] = pd.to_datetime(data['Time'])  # Convert 'Time' column to datetime
    analyze_data(data, output_file)
